In [1]:
!pip install -q accelerate PyPDF2 faiss-cpu nltk torch transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.6 MB/s eta 0:00:00


In [3]:
!pip install langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00


In [11]:
!pip install openai langchain gradio nltk faiss-cpu transformers


In [13]:
!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [18]:
import gradio as gr
import nltk
import faiss
import pandas as pd
from transformers import AutoModel
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Function to load models and resources
def load_resources():
    nltk.download('punkt')  # Ensure the Punkt tokenizer is downloaded
    # Load the embedding model
    model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
    # Load OpenAI LLM
    llm = ChatOpenAI(openai_api_key="your-openai-api-key", temperature=0.8)
    # Load text data and FAISS index
    df = pd.read_csv('med_assistant_db.csv')
    sentences = df['text'].tolist()
    faiss_index = faiss.read_index('med_assistant_index')
    return model, llm, sentences, faiss_index

# Initialize resources
model, llm, sentences, faiss_index = load_resources()

# Function to search in the FAISS index
def search_in_index(index, query, sentences, model, top_k=5, threshold=0.5):
    # Generate embeddings for the query
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().detach().numpy()
    # Perform FAISS search
    distances, indices = index.search(query_embedding, top_k)
    # Retrieve sentences and distances
    results = [(sentences[i], distances[0][j]) for j, i in enumerate(indices[0])]
    # Filter results based on a relevance threshold
    filtered_results = [res for res in results if res[1] < threshold]
    return filtered_results

# Function to craft the prompt and generate results
def craft_prompt_generate_results(input_str):
    # Refined prompt for better results
    template = """
    We are conducting an analysis to assist medical professionals by analyzing patient texts. This analysis is meant to complement, not replace, professional medical advice.

    If the input text does not indicate any concerning symptoms, state "No concerning symptoms identified," and do not include potential disorders or symptoms unless explicitly mentioned.

    Input Provided:
    {input_str}

    Your analysis should include:
    - Assessment: Clearly indicate whether there are concerning symptoms or not.
    - (Optional) Potential Disorders: Only list if specific symptoms indicate a disorder.
    - (Optional) Symptoms: Only list symptoms explicitly mentioned or implied in the text.
    - Recommendations: Provide general health recommendations based on the input, or state "No specific recommendations."

    Format your analysis in a clear, structured manner.
    """
    prompt_template = PromptTemplate(input_variables=["input_str"], template=template)
    question_chain = LLMChain(llm=llm, prompt=prompt_template)
    overall_chain = SimpleSequentialChain(chains=[question_chain])
    # Run the chain with the input string
    result = overall_chain.run(input_str)
    return result

# Function to clean up the output from the LLM
def post_process_output(output):
    if "No concerning symptoms identified" in output:
        # Remove unnecessary sections for this case
        if "- Potential Disorders:" in output:
            output = output.split("- Potential Disorders:")[0].strip()
    return output

# Gradio interface functions
def analyze_symptoms(symptoms):
    # Search in the FAISS index for related sentences
    results = search_in_index(faiss_index, symptoms, sentences, model)
    # Combine retrieved sentences into a single text
    text = ''.join([sentence for sentence, _ in results])
    # Prepare input for the LLM
    input_str = f"query: {symptoms}\nEnd of Query\ntext: {text}"
    # Generate results using the LLM
    result = craft_prompt_generate_results(input_str)
    # Post-process the LLM output
    processed_result = post_process_output(result)
    return processed_result

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# AI Med Assistant [Experiment]")
    gr.Markdown(
        "This tool assists in analyzing patient symptoms using OpenAI's GPT models. Please input symptoms to get a preliminary analysis."
    )
    # Input field for symptoms
    input_text = gr.Textbox(label="Enter Symptoms", placeholder="Type symptoms here...", lines=3)
    # Output field for analysis
    output_text = gr.Textbox(label="Analysis Result", placeholder="Results will appear here...", lines=10)
    # Button to trigger analysis
    analyze_button = gr.Button("Analyze Symptoms")
    # Link button to the analyze_symptoms function
    analyze_button.click(analyze_symptoms, inputs=input_text, outputs=output_text)

# Launch Gradio App
demo.launch()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bce3e8f2ea4c068eea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
import faiss
index = faiss.read_index('med_assistant_index')
print("Index Loaded:", index.is_trained)


Index Loaded: True


In [17]:
from transformers import AutoModel
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
print("Model Loaded Successfully")


Model Loaded Successfully


In [7]:
query = "My joints feel stiff and painful, especially in the morning."
query_embedding = model.encode([query], convert_to_tensor=True).cpu().detach().numpy()
distances, indices = index.search(query_embedding, k=5)
print("Search Results:", distances, indices)


Search Results: [[35.4064   41.269592 47.384407 60.73436  66.36656 ]] [[22340 21909 21928 21927 21881]]


In [ ]:
import openai

# Test OpenAI API connection
try:
    openai.api_key = "your-openai-api-key"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt="Test prompt",
        max_tokens=10
    )
    print("OpenAI API Working!")
except Exception as e:
    print(f"OpenAI API Error: {e}")
